In [1]:
# Import all Libraries
'''
Problem statement: Predict total fare for each flight for carriers L1, L2 and L3 on a given date

'''
import os
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
train_folder = './training/'
test_folder = './test/'

In [3]:
service_index_df = pd.read_csv(os.path.join(train_folder,'service_index.csv'))
train_schedules_df = pd.read_csv(os.path.join(train_folder,'train_schedules.csv'))
train_fares_df = pd.read_csv(os.path.join(train_folder,'train_fares.csv'))

In [4]:
# service_index_df = pd.read_csv(os.path.join(train_folder,'service_index.csv'))
test_schedules_df = pd.read_csv(os.path.join(test_folder,'test_schedules.csv'))
test_fares_df = pd.read_csv(os.path.join(test_folder,'test_fares_data.csv'))

In [5]:
service_index_df = service_index_df[service_index_df['carrier'].isin(['L1', 'L2','L3'])]
train_schedules_df = train_schedules_df[train_schedules_df['carrier'].isin(['L1', 'L2','L3'])]
train_fares_df = train_fares_df[train_fares_df['carrier'].isin(['L1', 'L2','L3'])]


In [6]:
# test dataset
test_schedules_df = test_schedules_df[test_schedules_df['carrier'].isin(['L1', 'L2','L3'])]
test_fares_df = test_fares_df[test_fares_df['carrier'].isin(['L1', 'L2','L3'])]

In [7]:
service_index_df.drop('Unnamed: 0',1,inplace=True)
train_schedules_df.drop('Unnamed: 0',1,inplace=True)
train_fares_df.drop('Unnamed: 0',1,inplace=True)

In [8]:
#test dataset

test_schedules_df.drop('Unnamed: 0',1,inplace=True)
test_fares_df.drop('Unnamed: 0',1,inplace=True)

In [9]:
# convert mm/dd/yyyy to yyyy-mm-dd
test_fares_df['observation_date']=test_fares_df['observation_date'].apply(lambda x :datetime.strptime(x, '%m/%d/%Y').strftime('%Y-%m-%d')) 
test_fares_df['flt_departure_dt']=test_fares_df['flt_departure_dt'].apply(lambda x :datetime.strptime(x, '%m/%d/%Y').strftime('%Y-%m-%d')) 

In [10]:
test_fares_df.dtypes

origin              object
destination         object
carrier             object
flt_num              int64
flt_departure_dt    object
observation_date    object
origin_city         object
destination_city    object
dtype: object

In [11]:
test_schedules_df.head()

,carrier,flt_num,origin,destination,flt_departure_dt,flt_departure_local_time,flt_arrival_local_time,flt_departure_gmt,flt_arrival_gmt
0,L1,3437,Airport26,Airport30,2019-01-01,2019-01-01 18:28:00.0,2019-01-01 19:49:00.0,2019-01-02 00:28:00.0,2019-01-02 03:49:00.0
1,L1,7760,Airport26,Airport30,2019-01-01,2019-01-01 07:50:00.0,2019-01-01 09:12:00.0,2019-01-01 13:50:00.0,2019-01-01 17:12:00.0
2,L1,1609,Airport26,Airport30,2019-01-01,2019-01-01 11:55:00.0,2019-01-01 13:19:00.0,2019-01-01 17:55:00.0,2019-01-01 21:19:00.0
3,L1,1487,Airport26,Airport30,2019-01-01,2019-01-01 14:20:00.0,2019-01-01 15:41:00.0,2019-01-01 20:20:00.0,2019-01-01 23:41:00.0
4,L1,5701,Airport26,Airport30,2019-01-01,2019-01-01 22:05:00.0,2019-01-01 23:26:00.0,2019-01-02 04:05:00.0,2019-01-02 07:26:00.0


In [12]:
service_index_df.head()

,yr,mo,origin,destination,carrier,scaled_demand,scaled_share
1,2017,12,Airport10,Airport43,L3,1708,169
2,2017,12,Airport10,Airport60,L2,1071,180
6,2017,12,Airport17,Airport37,L2,1639,160
9,2017,12,Airport17,Airport46,L2,2527,242
10,2017,12,Airport17,Airport51,L3,1287,170


In [13]:
train_schedules_df.head()

,carrier,flt_num,origin,destination,flt_departure_dt,flt_departure_local_time,flt_arrival_local_time,flt_departure_gmt,flt_arrival_gmt
0,L1,6800,Airport26,Airport30,2018-02-01,2018-02-01 18:43:00.0,2018-02-01 20:09:00.0,2018-02-02 00:43:00.0,2018-02-02 04:09:00.0
2,L1,799,Airport26,Airport30,2018-02-01,2018-02-01 12:00:00.0,2018-02-01 13:26:00.0,2018-02-01 18:00:00.0,2018-02-01 21:26:00.0
3,L1,7760,Airport26,Airport30,2018-02-01,2018-02-01 07:35:00.0,2018-02-01 09:02:00.0,2018-02-01 13:35:00.0,2018-02-01 17:02:00.0
4,L1,3443,Airport26,Airport30,2018-02-01,2018-02-01 09:48:00.0,2018-02-01 11:14:00.0,2018-02-01 15:48:00.0,2018-02-01 19:14:00.0
7,L1,6759,Airport26,Airport30,2018-02-01,2018-02-01 16:27:00.0,2018-02-01 17:54:00.0,2018-02-01 22:27:00.0,2018-02-02 01:54:00.0


In [14]:
train_fares_df.head()

,origin,destination,carrier,flt_num,flt_departure_dt,observation_date,total_fare,origin_city,destination_city
0,Airport4,Airport43,L1,5911,2018-01-03,2017-11-29,538,City4,City39
1,Airport4,Airport43,L1,6589,2018-01-03,2017-11-29,538,City4,City39
2,Airport4,Airport43,L1,2689,2018-01-03,2017-11-29,538,City4,City39
3,Airport4,Airport43,L2,8244,2018-01-03,2017-11-29,557,City4,City39
4,Airport4,Airport43,L2,8523,2018-01-03,2017-11-29,557,City4,City39


In [15]:
train_merged_df = pd.merge(train_fares_df,train_schedules_df, on=['flt_num','flt_departure_dt','carrier','origin','destination'])

In [16]:
#test dataset
test_merged_df = pd.merge(test_fares_df,test_schedules_df, on=['flt_num','flt_departure_dt','carrier','origin','destination'])

In [17]:
test_merged_df.head()

,origin,destination,carrier,flt_num,flt_departure_dt,observation_date,origin_city,destination_city,flt_departure_local_time,flt_arrival_local_time,flt_departure_gmt,flt_arrival_gmt
0,Airport4,Airport43,L2,7465,2019-01-01,2018-11-27,City4,City39,2019-01-01 08:04:00.0,2019-01-01 10:49:00.0,2019-01-01 14:04:00.0,2019-01-01 16:49:00.0
1,Airport4,Airport43,L2,7465,2019-01-01,2018-11-28,City4,City39,2019-01-01 08:04:00.0,2019-01-01 10:49:00.0,2019-01-01 14:04:00.0,2019-01-01 16:49:00.0
2,Airport4,Airport43,L2,7465,2019-01-01,2018-12-07,City4,City39,2019-01-01 08:04:00.0,2019-01-01 10:49:00.0,2019-01-01 14:04:00.0,2019-01-01 16:49:00.0
3,Airport4,Airport43,L2,7465,2019-01-01,2018-12-05,City4,City39,2019-01-01 08:04:00.0,2019-01-01 10:49:00.0,2019-01-01 14:04:00.0,2019-01-01 16:49:00.0
4,Airport4,Airport43,L2,7465,2019-01-01,2018-12-01,City4,City39,2019-01-01 08:04:00.0,2019-01-01 10:49:00.0,2019-01-01 14:04:00.0,2019-01-01 16:49:00.0


In [18]:
train_merged_df['departure_yr']=train_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[0]))
train_merged_df['departure_mo']=train_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[1]))
train_merged_df['departure_dt']=train_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[2]))

In [19]:
train_merged_df['yr']=train_merged_df['observation_date'].apply(lambda x: int(x.split('-')[0]))
train_merged_df['mo']=train_merged_df['observation_date'].apply(lambda x: int(x.split('-')[1]))
train_merged_df['dt']=train_merged_df['observation_date'].apply(lambda x: int(x.split('-')[2]))

In [20]:
# test dataset

test_merged_df['departure_yr']=test_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[0]))
test_merged_df['departure_mo']=test_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[1]))
test_merged_df['departure_dt']=test_merged_df['flt_departure_dt'].apply(lambda x: int(x.split('-')[2]))


test_merged_df['yr']=test_merged_df['observation_date'].apply(lambda x: int(x.split('-')[0]))
test_merged_df['mo']=test_merged_df['observation_date'].apply(lambda x: int(x.split('-')[1]))
test_merged_df['dt']=test_merged_df['observation_date'].apply(lambda x: int(x.split('-')[2]))

In [21]:
train_df = pd.merge(train_merged_df, service_index_df, on=['yr','mo','carrier','origin','destination'])

In [22]:
test_df = pd.merge(test_merged_df, service_index_df, on=['yr','mo','carrier','origin','destination'])

In [25]:

train_df['flt_departure_local_time']=train_df['flt_departure_local_time'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))

train_df['flt_arrival_local_time']=train_df['flt_arrival_local_time'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))

train_df['flt_departure_gmt']=train_df['flt_departure_gmt'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
                                                                                
train_df['flt_arrival_gmt']=train_df['flt_arrival_gmt'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
                                                                                  

In [26]:
# test dataset


test_df['flt_departure_local_time']=test_df['flt_departure_local_time'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))

test_df['flt_arrival_local_time']=test_df['flt_arrival_local_time'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))

test_df['flt_departure_gmt']=test_df['flt_departure_gmt'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
                                                                                
test_df['flt_arrival_gmt']=test_df['flt_arrival_gmt'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
                                                                                  

In [27]:
train_df['flight_travel_duration'] = train_df['flt_arrival_gmt']-train_df['flt_departure_gmt']

In [28]:
train_df['flight_travel_duration'] = train_df['flight_travel_duration'].apply(lambda x: x.seconds/3600)

In [29]:
# test dataset
test_df['flight_travel_duration'] = test_df['flt_arrival_gmt']-test_df['flt_departure_gmt']
test_df['flight_travel_duration'] = test_df['flight_travel_duration'].apply(lambda x: x.seconds/3600)

In [32]:
def get_part_of_day(hour):
    return (
    "morning" if 5 <= hour <= 11
    else
    "afternoon" if 12 <= hour <= 17
    else
    "evening" if 18 <= hour <= 22
    else
    "night"
        )

In [33]:
train_df['_flt_departure_local_time'] = train_df.flt_departure_local_time.apply(lambda x: get_part_of_day(x.hour))
train_df['_flt_arrival_local_time'] = train_df.flt_arrival_local_time.apply(lambda x: get_part_of_day(x.hour))
train_df['_flt_departure_gmt'] = train_df.flt_departure_gmt.apply(lambda x: get_part_of_day(x.hour))
train_df['_flt_arrival_gmt'] = train_df.flt_arrival_gmt.apply(lambda x: get_part_of_day(x.hour))

In [34]:
# test dataset

test_df['_flt_departure_local_time'] = test_df.flt_departure_local_time.apply(lambda x: get_part_of_day(x.hour))
test_df['_flt_arrival_local_time'] = test_df.flt_arrival_local_time.apply(lambda x: get_part_of_day(x.hour))
test_df['_flt_departure_gmt'] = test_df.flt_departure_gmt.apply(lambda x: get_part_of_day(x.hour))
test_df['_flt_arrival_gmt'] = test_df.flt_arrival_gmt.apply(lambda x: get_part_of_day(x.hour))

In [35]:
train_df['flt_departure_dt']=train_df['flt_departure_dt'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))

train_df['observation_date']=train_df['observation_date'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))

test_df['flt_departure_dt']=test_df['flt_departure_dt'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))

test_df['observation_date']=test_df['observation_date'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))

In [36]:
train_df['flt_departure_dayofyear'] = train_df.flt_departure_dt.dt.dayofyear
train_df['flt_departure_weekday'] = train_df.flt_departure_dt.dt.weekday
train_df['flt_departure_week'] = train_df.flt_departure_dt.dt.week
train_df['flt_departure_quarter'] = train_df.flt_departure_dt.dt.quarter

train_df['observation_date_dayofyear'] = train_df.observation_date.dt.dayofyear
train_df['observation_date_weekday'] = train_df.observation_date.dt.weekday
train_df['observation_date_week'] = train_df.observation_date.dt.week
train_df['observation_date_quarter'] = train_df.observation_date.dt.quarter

# test dataset
test_df['flt_departure_dayofyear'] = test_df.flt_departure_dt.dt.dayofyear
test_df['flt_departure_weekday'] = test_df.flt_departure_dt.dt.weekday
test_df['flt_departure_week'] = test_df.flt_departure_dt.dt.week
test_df['flt_departure_quarter'] = test_df.flt_departure_dt.dt.quarter

test_df['observation_date_dayofyear'] = test_df.observation_date.dt.dayofyear
test_df['observation_date_weekday'] = test_df.observation_date.dt.weekday
test_df['observation_date_week'] = test_df.observation_date.dt.week
test_df['observation_date_quarter'] = test_df.observation_date.dt.quarter

In [37]:
cols_to_drop = ['flt_departure_dt','observation_date','flt_departure_local_time','flt_arrival_local_time',
                'flt_departure_gmt','flt_arrival_gmt']

In [38]:
[train_df.drop(col,1,inplace=True) for col in cols_to_drop]

[None, None, None, None, None, None]

In [39]:
# test dataset
[test_df.drop(col,1,inplace=True) for col in cols_to_drop]

[None, None, None, None, None, None]

In [42]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [43]:
# test dataset
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [44]:
target = train_df['total_fare']
train_df.drop('total_fare',1,inplace=True)

In [45]:
X = pd.get_dummies(train_df, prefix_sep='_', drop_first=True)

In [46]:
# test dataset
X_ = pd.get_dummies(test_df, prefix_sep='_', drop_first=True)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.1, random_state=42)

In [49]:
baseline_model_cb = CatBoostRegressor(iterations=5000,learning_rate=0.5)
baseline_model_cb.fit(X_train,y_train,eval_set=(X_test,y_test),plot=True,use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 455.7672955	test: 457.4186023	best: 457.4186023 (0)	total: 258ms	remaining: 21m 30s
1:	learn: 436.8176282	test: 438.7158915	best: 438.7158915 (1)	total: 408ms	remaining: 16m 58s
2:	learn: 428.6745498	test: 430.4573209	best: 430.4573209 (2)	total: 551ms	remaining: 15m 18s
3:	learn: 421.7220183	test: 423.5867962	best: 423.5867962 (3)	total: 727ms	remaining: 15m 8s
4:	learn: 418.8974455	test: 420.5700513	best: 420.5700513 (4)	total: 846ms	remaining: 14m 5s
5:	learn: 415.3985062	test: 417.0455243	best: 417.0455243 (5)	total: 960ms	remaining: 13m 19s
6:	learn: 413.0665489	test: 414.6497861	best: 414.6497861 (6)	total: 1.09s	remaining: 12m 58s
7:	learn: 408.1778430	test: 410.1965302	best: 410.1965302 (7)	total: 1.29s	remaining: 13m 23s
8:	learn: 405.9692564	test: 407.8749420	best: 407.8749420 (8)	total: 1.43s	remaining: 13m 12s
9:	learn: 403.1560245	test: 405.0167004	best: 405.0167004 (9)	total: 1.54s	remaining: 12m 50s
10:	learn: 399.9579320	test: 401.6603798	best: 401.6603798 (10

86:	learn: 317.0950126	test: 320.3687877	best: 320.3687877 (86)	total: 12.1s	remaining: 11m 23s
87:	learn: 316.7088253	test: 319.9711633	best: 319.9711633 (87)	total: 12.5s	remaining: 11m 35s
88:	learn: 316.4630892	test: 319.6685929	best: 319.6685929 (88)	total: 12.6s	remaining: 11m 34s
89:	learn: 315.9587089	test: 319.1564980	best: 319.1564980 (89)	total: 12.7s	remaining: 11m 35s
90:	learn: 315.6979072	test: 318.9365683	best: 318.9365683 (90)	total: 13s	remaining: 11m 40s
91:	learn: 315.1754820	test: 318.4486830	best: 318.4486830 (91)	total: 13.2s	remaining: 11m 43s
92:	learn: 314.6385132	test: 317.9463268	best: 317.9463268 (92)	total: 13.4s	remaining: 11m 46s
93:	learn: 314.0232086	test: 317.0958103	best: 317.0958103 (93)	total: 13.5s	remaining: 11m 45s
94:	learn: 313.4198477	test: 316.3526505	best: 316.3526505 (94)	total: 13.7s	remaining: 11m 45s
95:	learn: 313.1917994	test: 316.1338585	best: 316.1338585 (95)	total: 13.9s	remaining: 11m 47s
96:	learn: 312.6920384	test: 315.6129979	b

172:	learn: 293.1440340	test: 296.3058075	best: 296.3058075 (172)	total: 25.8s	remaining: 11m 59s
173:	learn: 292.9956330	test: 296.1426764	best: 296.1426764 (173)	total: 25.9s	remaining: 11m 58s
174:	learn: 292.8894318	test: 296.0274848	best: 296.0274848 (174)	total: 26s	remaining: 11m 58s
175:	learn: 292.7169266	test: 295.8669265	best: 295.8669265 (175)	total: 26.2s	remaining: 11m 58s
176:	learn: 292.4049889	test: 295.5745260	best: 295.5745260 (176)	total: 26.3s	remaining: 11m 57s
177:	learn: 292.2339308	test: 295.4308974	best: 295.4308974 (177)	total: 26.5s	remaining: 11m 58s
178:	learn: 292.1601003	test: 295.3626568	best: 295.3626568 (178)	total: 26.7s	remaining: 11m 57s
179:	learn: 292.0000300	test: 295.2146387	best: 295.2146387 (179)	total: 26.8s	remaining: 11m 57s
180:	learn: 291.7804143	test: 294.9833335	best: 294.9833335 (180)	total: 26.9s	remaining: 11m 57s
181:	learn: 291.5943999	test: 294.8028677	best: 294.8028677 (181)	total: 27.1s	remaining: 11m 56s
182:	learn: 291.413500

257:	learn: 280.3784328	test: 283.4343981	best: 283.4343981 (257)	total: 39.7s	remaining: 12m 9s
258:	learn: 280.2329351	test: 283.2993646	best: 283.2993646 (258)	total: 39.9s	remaining: 12m 9s
259:	learn: 280.1253680	test: 283.2047099	best: 283.2047099 (259)	total: 40s	remaining: 12m 9s
260:	learn: 280.0458422	test: 283.1374997	best: 283.1374997 (260)	total: 40.2s	remaining: 12m 9s
261:	learn: 279.9263312	test: 283.0265414	best: 283.0265414 (261)	total: 40.4s	remaining: 12m 10s
262:	learn: 279.7998447	test: 282.8991444	best: 282.8991444 (262)	total: 40.6s	remaining: 12m 10s
263:	learn: 279.7416941	test: 282.8328017	best: 282.8328017 (263)	total: 40.7s	remaining: 12m 10s
264:	learn: 279.6023235	test: 282.6523478	best: 282.6523478 (264)	total: 40.9s	remaining: 12m 10s
265:	learn: 279.4829158	test: 282.5088279	best: 282.5088279 (265)	total: 41.1s	remaining: 12m 11s
266:	learn: 279.3567142	test: 282.4130329	best: 282.4130329 (266)	total: 41.3s	remaining: 12m 11s
267:	learn: 279.1369384	te

...

4665:	learn: 190.7676220	test: 207.5857628	best: 207.5857628 (4665)	total: 14m 22s	remaining: 1m 1s
4666:	learn: 190.7617975	test: 207.5893102	best: 207.5857628 (4665)	total: 14m 23s	remaining: 1m 1s
4667:	learn: 190.7338611	test: 207.5875009	best: 207.5857628 (4665)	total: 14m 23s	remaining: 1m 1s
4668:	learn: 190.7209626	test: 207.5945682	best: 207.5857628 (4665)	total: 14m 23s	remaining: 1m 1s
4669:	learn: 190.7077908	test: 207.6077483	best: 207.5857628 (4665)	total: 14m 23s	remaining: 1m 1s
4670:	learn: 190.7018222	test: 207.6028156	best: 207.5857628 (4665)	total: 14m 23s	remaining: 1m
4671:	learn: 190.7002800	test: 207.6000985	best: 207.5857628 (4665)	total: 14m 24s	remaining: 1m
4672:	learn: 190.6950012	test: 207.5974090	best: 207.5857628 (4665)	total: 14m 24s	remaining: 1m
4673:	learn: 190.6898177	test: 207.5974545	best: 207.5857628 (4665)	total: 14m 24s	remaining: 1m
4674:	learn: 190.6859370	test: 207.5978507	best: 207.5857628 (4665)	total: 14m 24s	remaining: 1m
4675:	learn: 19

4748:	learn: 190.1408476	test: 207.2448919	best: 207.2448919 (4748)	total: 14m 39s	remaining: 46.5s
4749:	learn: 190.1170116	test: 207.2452255	best: 207.2448919 (4748)	total: 14m 39s	remaining: 46.3s
4750:	learn: 190.1153474	test: 207.2457813	best: 207.2448919 (4748)	total: 14m 39s	remaining: 46.1s
4751:	learn: 190.1035770	test: 207.2542216	best: 207.2448919 (4748)	total: 14m 39s	remaining: 45.9s
4752:	learn: 190.1014040	test: 207.2534777	best: 207.2448919 (4748)	total: 14m 40s	remaining: 45.7s
4753:	learn: 190.0969377	test: 207.2606383	best: 207.2448919 (4748)	total: 14m 40s	remaining: 45.5s
4754:	learn: 190.0891621	test: 207.2462584	best: 207.2448919 (4748)	total: 14m 40s	remaining: 45.4s
4755:	learn: 190.0827345	test: 207.2457205	best: 207.2448919 (4748)	total: 14m 40s	remaining: 45.2s
4756:	learn: 190.0780227	test: 207.2445851	best: 207.2445851 (4756)	total: 14m 40s	remaining: 45s
4757:	learn: 190.0706351	test: 207.2414886	best: 207.2414886 (4757)	total: 14m 41s	remaining: 44.8s
47

4831:	learn: 189.5537139	test: 206.9126080	best: 206.8934485 (4826)	total: 14m 55s	remaining: 31.1s
4832:	learn: 189.5457992	test: 206.9011760	best: 206.8934485 (4826)	total: 14m 55s	remaining: 31s
4833:	learn: 189.5333956	test: 206.9047648	best: 206.8934485 (4826)	total: 14m 55s	remaining: 30.8s
4834:	learn: 189.5313065	test: 206.9025928	best: 206.8934485 (4826)	total: 14m 56s	remaining: 30.6s
4835:	learn: 189.5254250	test: 206.8960149	best: 206.8934485 (4826)	total: 14m 56s	remaining: 30.4s
4836:	learn: 189.5190963	test: 206.8932095	best: 206.8932095 (4836)	total: 14m 56s	remaining: 30.2s
4837:	learn: 189.5146098	test: 206.8907011	best: 206.8907011 (4837)	total: 14m 56s	remaining: 30s
4838:	learn: 189.5108489	test: 206.8885859	best: 206.8885859 (4838)	total: 14m 56s	remaining: 29.8s
4839:	learn: 189.5082417	test: 206.8863103	best: 206.8863103 (4839)	total: 14m 56s	remaining: 29.6s
4840:	learn: 189.5036639	test: 206.8835709	best: 206.8835709 (4840)	total: 14m 57s	remaining: 29.5s
4841

4915:	learn: 189.0575551	test: 206.6370624	best: 206.6370624 (4915)	total: 15m 11s	remaining: 15.6s
4916:	learn: 189.0227889	test: 206.6549834	best: 206.6370624 (4915)	total: 15m 11s	remaining: 15.4s
4917:	learn: 188.9993903	test: 206.6285926	best: 206.6285926 (4917)	total: 15m 11s	remaining: 15.2s
4918:	learn: 188.9841417	test: 206.6324493	best: 206.6285926 (4917)	total: 15m 12s	remaining: 15s
4919:	learn: 188.9800691	test: 206.6278644	best: 206.6278644 (4919)	total: 15m 12s	remaining: 14.8s
4920:	learn: 188.9762190	test: 206.6257686	best: 206.6257686 (4920)	total: 15m 12s	remaining: 14.6s
4921:	learn: 188.9730170	test: 206.6246860	best: 206.6246860 (4921)	total: 15m 12s	remaining: 14.5s
4922:	learn: 188.9668106	test: 206.6157817	best: 206.6157817 (4922)	total: 15m 12s	remaining: 14.3s
4923:	learn: 188.9626536	test: 206.6168384	best: 206.6157817 (4922)	total: 15m 12s	remaining: 14.1s
4924:	learn: 188.9577016	test: 206.6157708	best: 206.6157708 (4924)	total: 15m 13s	remaining: 13.9s
49

4999:	learn: 188.4773446	test: 206.2382737	best: 206.2382737 (4999)	total: 15m 27s	remaining: 0us

bestTest = 206.2382737
bestIteration = 4999



In [50]:
y_pred_baseline = baseline_model_cb.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_baseline))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_baseline))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_baseline)))

Mean Absolute Error: 112.73783946762764
Mean Squared Error: 42534.225545853944
Root Mean Squared Error: 206.23827371720785


In [59]:
grid = {'learning_rate': [0.05, 0.3, 0.5],
        'depth': [6, 10, 12],
        'l2_leaf_reg': [1, 1.5 , 2.5, 3]}

In [60]:
grid_model = CatBoostRegressor()

In [62]:
grid_search_result = grid_model.grid_search(grid, 
                                       X=X, 
                                       y=target, 
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	loss: 312.1043031	best: 312.1043031 (0)	total: 2m 53s	remaining: 1h 41m 13s
1:	loss: 256.0642636	best: 256.0642636 (1)	total: 4m 50s	remaining: 1h 22m 15s
2:	loss: 246.1419928	best: 246.1419928 (2)	total: 6m 46s	remaining: 1h 14m 34s
3:	loss: 311.2332978	best: 246.1419928 (2)	total: 8m 44s	remaining: 1h 9m 57s
4:	loss: 255.6909508	best: 246.1419928 (2)	total: 10m 41s	remaining: 1h 6m 19s
5:	loss: 245.0132007	best: 245.0132007 (5)	total: 12m 39s	remaining: 1h 3m 16s
6:	loss: 311.8733702	best: 245.0132007 (5)	total: 14m 37s	remaining: 1h 35s
7:	loss: 255.8570919	best: 245.0132007 (5)	total: 16m 34s	remaining: 58m 1s
8:	loss: 245.4733046	best: 245.0132007 (5)	total: 18m 33s	remaining: 55m 39s
9:	loss: 312.4197315	best: 245.0132007 (5)	total: 20m 31s	remaining: 53m 21s
10:	loss: 255.7435646	best: 245.0132007 (5)	total: 22m 28s	remaining: 51m 4s
11:	loss: 245.9424085	best: 245.0132007 (5)	total: 24m 26s	remaining: 48m 53s
12:	loss: 262.7575776	best: 245.0132007 (5)	total: 29m 11s	remaini

In [63]:
grid_search_result['params'] #{'depth': 12, 'l2_leaf_reg': 1, 'learning_rate': 0.5}

{'depth': 12, 'l2_leaf_reg': 1, 'learning_rate': 0.5}

In [51]:
grid_model = CatBoostRegressor(depth= 12, l2_leaf_reg= 1, learning_rate= 0.5, iterations= 5000,early_stopping_rounds=50)

In [52]:
grid_model.fit(X_train,y_train,eval_set=(X_test,y_test) , plot=True,use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 440.3813666	test: 442.1597663	best: 442.1597663 (0)	total: 519ms	remaining: 43m 15s
1:	learn: 415.6282650	test: 417.5564259	best: 417.5564259 (1)	total: 1.01s	remaining: 42m
2:	learn: 397.7762720	test: 399.7084639	best: 399.7084639 (2)	total: 1.59s	remaining: 44m 10s
3:	learn: 388.9483667	test: 390.7337327	best: 390.7337327 (3)	total: 2.13s	remaining: 44m 24s
4:	learn: 380.3875980	test: 382.3312777	best: 382.3312777 (4)	total: 2.76s	remaining: 45m 59s
5:	learn: 372.3323417	test: 374.4345412	best: 374.4345412 (5)	total: 3.32s	remaining: 46m 4s
6:	learn: 365.0256817	test: 367.0506459	best: 367.0506459 (6)	total: 3.9s	remaining: 46m 21s
7:	learn: 361.0573414	test: 363.1966879	best: 363.1966879 (7)	total: 4.47s	remaining: 46m 32s
8:	learn: 353.7315389	test: 355.7816683	best: 355.7816683 (8)	total: 4.97s	remaining: 45m 54s
9:	learn: 349.0867817	test: 351.0477083	best: 351.0477083 (9)	total: 5.58s	remaining: 46m 23s
10:	learn: 345.5248651	test: 347.6997021	best: 347.6997021 (10)	to

87:	learn: 245.5875146	test: 253.5222259	best: 253.5222259 (87)	total: 47.2s	remaining: 43m 55s
88:	learn: 244.5452987	test: 252.6301559	best: 252.6301559 (88)	total: 47.7s	remaining: 43m 54s
89:	learn: 244.0560079	test: 252.2505849	best: 252.2505849 (89)	total: 48.2s	remaining: 43m 51s
90:	learn: 243.3127422	test: 251.7231374	best: 251.7231374 (90)	total: 48.8s	remaining: 43m 50s
91:	learn: 242.8198357	test: 251.4006849	best: 251.4006849 (91)	total: 49.3s	remaining: 43m 47s
92:	learn: 242.2654280	test: 250.9072542	best: 250.9072542 (92)	total: 49.8s	remaining: 43m 45s
93:	learn: 241.8407952	test: 250.5728264	best: 250.5728264 (93)	total: 50.3s	remaining: 43m 46s
94:	learn: 241.1644111	test: 250.0440673	best: 250.0440673 (94)	total: 50.9s	remaining: 43m 46s
95:	learn: 240.7842380	test: 249.6972556	best: 249.6972556 (95)	total: 51.4s	remaining: 43m 44s
96:	learn: 240.2120486	test: 249.2243665	best: 249.2243665 (96)	total: 52s	remaining: 43m 46s
97:	learn: 239.9275221	test: 248.9544325	b

171:	learn: 214.0980170	test: 227.4040686	best: 227.4040686 (171)	total: 1m 34s	remaining: 44m 24s
172:	learn: 213.7532811	test: 227.1998622	best: 227.1998622 (172)	total: 1m 35s	remaining: 44m 23s
173:	learn: 213.5187513	test: 227.0043388	best: 227.0043388 (173)	total: 1m 36s	remaining: 44m 26s
174:	learn: 213.2415025	test: 226.7491426	best: 226.7491426 (174)	total: 1m 36s	remaining: 44m 34s
175:	learn: 212.9721389	test: 226.5238915	best: 226.5238915 (175)	total: 1m 37s	remaining: 44m 33s
176:	learn: 212.7847100	test: 226.3431693	best: 226.3431693 (176)	total: 1m 38s	remaining: 44m 31s
177:	learn: 212.5148697	test: 226.1821890	best: 226.1821890 (177)	total: 1m 38s	remaining: 44m 33s
178:	learn: 212.2984231	test: 225.9618280	best: 225.9618280 (178)	total: 1m 39s	remaining: 44m 31s
179:	learn: 212.1360199	test: 225.8601608	best: 225.8601608 (179)	total: 1m 39s	remaining: 44m 30s
180:	learn: 211.9315737	test: 225.6493254	best: 225.6493254 (180)	total: 1m 40s	remaining: 44m 39s
181:	learn

255:	learn: 196.5505535	test: 214.7234884	best: 214.7234884 (255)	total: 2m 24s	remaining: 44m 42s
256:	learn: 196.3439477	test: 214.5889027	best: 214.5889027 (256)	total: 2m 25s	remaining: 44m 40s
257:	learn: 196.1523451	test: 214.3922604	best: 214.3922604 (257)	total: 2m 25s	remaining: 44m 38s
258:	learn: 195.9952043	test: 214.2277981	best: 214.2277981 (258)	total: 2m 26s	remaining: 44m 37s
259:	learn: 195.8242662	test: 214.0977357	best: 214.0977357 (259)	total: 2m 26s	remaining: 44m 35s
260:	learn: 195.6457023	test: 214.0026878	best: 214.0026878 (260)	total: 2m 27s	remaining: 44m 34s
261:	learn: 195.4447338	test: 213.8530902	best: 213.8530902 (261)	total: 2m 27s	remaining: 44m 32s
262:	learn: 195.2181249	test: 213.6676547	best: 213.6676547 (262)	total: 2m 28s	remaining: 44m 30s
263:	learn: 195.0600841	test: 213.5512994	best: 213.5512994 (263)	total: 2m 28s	remaining: 44m 29s
264:	learn: 194.8423594	test: 213.4464418	best: 213.4464418 (264)	total: 2m 29s	remaining: 44m 27s
265:	learn

...

2629:	learn: 98.4241586	test: 169.5245310	best: 169.4799283 (2616)	total: 38m 27s	remaining: 34m 39s
2630:	learn: 98.4163361	test: 169.5263807	best: 169.4799283 (2616)	total: 38m 29s	remaining: 34m 39s
2631:	learn: 98.4030758	test: 169.5269105	best: 169.4799283 (2616)	total: 38m 31s	remaining: 34m 39s
2632:	learn: 98.3827164	test: 169.5296243	best: 169.4799283 (2616)	total: 38m 33s	remaining: 34m 39s
2633:	learn: 98.3644878	test: 169.5435128	best: 169.4799283 (2616)	total: 38m 34s	remaining: 34m 39s
2634:	learn: 98.3406715	test: 169.5443496	best: 169.4799283 (2616)	total: 38m 36s	remaining: 34m 39s
2635:	learn: 98.3239614	test: 169.5347677	best: 169.4799283 (2616)	total: 38m 38s	remaining: 34m 39s
2636:	learn: 98.3038193	test: 169.5196757	best: 169.4799283 (2616)	total: 38m 40s	remaining: 34m 39s
2637:	learn: 98.2892776	test: 169.5290818	best: 169.4799283 (2616)	total: 38m 42s	remaining: 34m 39s
2638:	learn: 98.2789063	test: 169.5303768	best: 169.4799283 (2616)	total: 38m 44s	remaining

2711:	learn: 97.2794218	test: 169.2251721	best: 169.2224104 (2709)	total: 41m 2s	remaining: 34m 37s
2712:	learn: 97.2695152	test: 169.2237673	best: 169.2224104 (2709)	total: 41m 3s	remaining: 34m 37s
2713:	learn: 97.2576837	test: 169.2142977	best: 169.2142977 (2713)	total: 41m 5s	remaining: 34m 37s
2714:	learn: 97.2464409	test: 169.2162183	best: 169.2142977 (2713)	total: 41m 7s	remaining: 34m 37s
2715:	learn: 97.2403002	test: 169.2116653	best: 169.2116653 (2715)	total: 41m 9s	remaining: 34m 36s
2716:	learn: 97.2221194	test: 169.2157238	best: 169.2116653 (2715)	total: 41m 11s	remaining: 34m 36s
2717:	learn: 97.2142706	test: 169.2140073	best: 169.2116653 (2715)	total: 41m 13s	remaining: 34m 36s
2718:	learn: 97.1794778	test: 169.1926344	best: 169.1926344 (2718)	total: 41m 15s	remaining: 34m 36s
2719:	learn: 97.1672136	test: 169.1848051	best: 169.1848051 (2719)	total: 41m 16s	remaining: 34m 36s
2720:	learn: 97.1552571	test: 169.1788173	best: 169.1788173 (2720)	total: 41m 18s	remaining: 34m

2793:	learn: 96.2863511	test: 168.9926355	best: 168.9926355 (2793)	total: 44m 58s	remaining: 35m 30s
2794:	learn: 96.2796157	test: 168.9903573	best: 168.9903573 (2794)	total: 45m 17s	remaining: 35m 43s
2795:	learn: 96.2636226	test: 168.9864623	best: 168.9864623 (2795)	total: 45m 23s	remaining: 35m 46s
2796:	learn: 96.2590386	test: 168.9867706	best: 168.9864623 (2795)	total: 45m 23s	remaining: 35m 45s
2797:	learn: 96.2516570	test: 168.9882047	best: 168.9864623 (2795)	total: 45m 24s	remaining: 35m 44s
2798:	learn: 96.2414531	test: 168.9842959	best: 168.9842959 (2798)	total: 45m 25s	remaining: 35m 42s
2799:	learn: 96.2367811	test: 168.9792595	best: 168.9792595 (2799)	total: 45m 25s	remaining: 35m 41s
2800:	learn: 96.2290922	test: 168.9750896	best: 168.9750896 (2800)	total: 45m 28s	remaining: 35m 42s
2801:	learn: 96.2227959	test: 168.9731123	best: 168.9731123 (2801)	total: 45m 29s	remaining: 35m 40s
2802:	learn: 96.2114495	test: 168.9796053	best: 168.9731123 (2801)	total: 45m 30s	remaining

2875:	learn: 95.3068374	test: 168.8392003	best: 168.8392003 (2875)	total: 46m 9s	remaining: 34m 5s
2876:	learn: 95.2928881	test: 168.8464275	best: 168.8392003 (2875)	total: 46m 9s	remaining: 34m 3s
2877:	learn: 95.2832501	test: 168.8421988	best: 168.8392003 (2875)	total: 46m 10s	remaining: 34m 2s
2878:	learn: 95.2618752	test: 168.8301399	best: 168.8301399 (2878)	total: 46m 10s	remaining: 34m 1s
2879:	learn: 95.2458656	test: 168.8283838	best: 168.8283838 (2879)	total: 46m 11s	remaining: 33m 59s
2880:	learn: 95.2181280	test: 168.8282349	best: 168.8282349 (2880)	total: 46m 11s	remaining: 33m 58s
2881:	learn: 95.2139972	test: 168.8247946	best: 168.8247946 (2881)	total: 46m 12s	remaining: 33m 57s
2882:	learn: 95.1915723	test: 168.8236351	best: 168.8236351 (2882)	total: 46m 12s	remaining: 33m 56s
2883:	learn: 95.1842749	test: 168.8280605	best: 168.8236351 (2882)	total: 46m 13s	remaining: 33m 54s
2884:	learn: 95.1754366	test: 168.8264187	best: 168.8236351 (2882)	total: 46m 13s	remaining: 33m 

2957:	learn: 94.3669012	test: 168.6099675	best: 168.6099675 (2957)	total: 46m 52s	remaining: 32m 21s
2958:	learn: 94.3620961	test: 168.6103110	best: 168.6099675 (2957)	total: 46m 53s	remaining: 32m 20s
2959:	learn: 94.3536459	test: 168.6162042	best: 168.6099675 (2957)	total: 46m 54s	remaining: 32m 19s
2960:	learn: 94.3405887	test: 168.6130139	best: 168.6099675 (2957)	total: 46m 54s	remaining: 32m 18s
2961:	learn: 94.3325333	test: 168.6127132	best: 168.6099675 (2957)	total: 46m 55s	remaining: 32m 16s
2962:	learn: 94.3125642	test: 168.6141225	best: 168.6099675 (2957)	total: 46m 55s	remaining: 32m 15s
2963:	learn: 94.3026115	test: 168.6095320	best: 168.6095320 (2963)	total: 46m 56s	remaining: 32m 14s
2964:	learn: 94.2917152	test: 168.6082190	best: 168.6082190 (2964)	total: 46m 56s	remaining: 32m 13s
2965:	learn: 94.2858231	test: 168.6079356	best: 168.6079356 (2965)	total: 46m 57s	remaining: 32m 11s
2966:	learn: 94.2661117	test: 168.6007141	best: 168.6007141 (2966)	total: 46m 57s	remaining

In [53]:
y_pred = grid_model.predict(X_test)

In [54]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 83.58818848626281
Mean Squared Error: 28422.813623058762
Root Mean Squared Error: 168.59066884931315


In [56]:
feature_imp_df = pd.DataFrame()
feature_imp_df['feature_name'] = grid_model.feature_names_
feature_imp_df['feature_importance']= grid_model.get_feature_importance()

In [58]:
feature_imp_df.iplot(x='feature_name',y='feature_importance',kind='bar',title='Feature Importance Plot',xTitle='Feature Name',yTitle='Importance')

In [61]:
grid_model.save_model('cb_best')